## Кривоногов Н.В., NLP, практическое задание № 9

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор

https://drive.google.com/file/d/11RozsaDlKoB8-7MKrS-gG1YfwruPob-Q/view?usp=sharing

In [1]:
! gdown 11RozsaDlKoB8-7MKrS-gG1YfwruPob-Q

Downloading...
From: https://drive.google.com/uc?id=11RozsaDlKoB8-7MKrS-gG1YfwruPob-Q
To: /content/GB_NLP_HW_9.zip
100% 305k/305k [00:00<00:00, 135MB/s]


In [2]:
! unzip -o /content/GB_NLP_HW_9.zip

Archive:  /content/GB_NLP_HW_9.zip
  inflating: 1984.txt                
  inflating: TheAgeofStillbirths.txt  


In [3]:
import tensorflow as tf

import numpy as np
import os
import time

Обучение модели (слои LSTM) на произведении "Эпоха мертворожденных" писателя Глеба Боброва (русский текст):

In [4]:
path_to_file = '/content/TheAgeofStillbirths.txt'

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='windows-1251')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 162718 characters


In [6]:
print(text[:500])

Эпоха мертворожденных. Украина в крови
Глеб Леонидович Бобров


Враг у ворот. Фантастика ближнего боя
Эту книгу не зря величают пророческой. Этот бестселлер, впервые опубликованный еще 8 лет назад, с поразительной точностью предсказал нынешнюю войну на Украине, зверства киевских карателей и героическое сопротивление Новороссии – так что язык не поворачивается назвать роман «фантастическим». Это больше, чем просто фантастика. Глеб Бобров, сам бывший «афганец», знает изнанку войны не понаслыш


In [7]:
text = text + text

In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

134 unique characters


In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
text_as_int, text, len(text_as_int), len(text)

(array([ 96, 114, 113, ...,   1,   2,   1]),
 'Эпоха мертворожденных. Украина в крови\r\nГлеб Леонидович Бобров\r\n\r\n\r\nВраг у ворот. Фантастика ближнего боя\r\nЭту книгу не зря величают пророческой. Этот бестселлер, впервые опубликованный еще 8 лет назад, с поразительной точностью предсказал нынешнюю войну на Украине, зверства киевских карателей и героическое сопротивление Новороссии – так что язык не поворачивается назвать роман «фантастическим». Это больше, чем просто фантастика. Глеб Бобров, сам бывший «афганец», знает изнанку войны не понаслышке. Только ветеран и мог написать такую книгу – настолько мощно и достоверно, с такими подробностями боевой работы и диверсионной борьбы, с таким натурализмом и полным погружением в кровавый кошмар бандеровского геноцида.\r\n\r\nИ не обольщайтесь. Этот роман – не об Украине. Пришествие «эпохи мертворожденных» грозит всем нам. После Новороссии на очереди – Россия. «Поэтому не спрашивай, по ком звонит колокол,\xa0– он звонит по тебе».\r\n\r\

### train and target

In [11]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

Э
п
о
х
а


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Эпоха мертворожденных. Украина в крови\r\nГлеб Леонидович Бобров\r\n\r\n\r\nВраг у ворот. Фантастика ближнего'
' боя\r\nЭту книгу не зря величают пророческой. Этот бестселлер, впервые опубликованный еще 8 лет назад,'
' с поразительной точностью предсказал нынешнюю войну на Украине, зверства киевских карателей и героич'
'еское сопротивление Новороссии – так что язык не поворачивается назвать роман «фантастическим». Это б'
'ольше, чем просто фантастика. Глеб Бобров, сам бывший «афганец», знает изнанку войны не понаслышке. Т'


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [14]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Эпоха мертворожденных. Украина в крови\r\nГлеб Леонидович Бобров\r\n\r\n\r\nВраг у ворот. Фантастика ближнег'
Target data: 'поха мертворожденных. Украина в крови\r\nГлеб Леонидович Бобров\r\n\r\n\r\nВраг у ворот. Фантастика ближнего'


In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [17]:
# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Embedding(vocab_size, embedding_dim),

#         tf.keras.layers.GRU(rnn_units,
#                             return_sequences=True,
#                             stateful=False,
#                             recurrent_initializer='glorot_uniform'),

#         tf.keras.layers.GRU(rnn_units,
#                             return_sequences=True,
#                             stateful=False,
#                             recurrent_initializer='glorot_uniform'),

#          tf.keras.layers.GRU(rnn_units,
#                             return_sequences=True,
#                             stateful=False,
#                             recurrent_initializer='glorot_uniform'),

#         tf.keras.layers.Dense(vocab_size)
#     ])
#     return model


def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [18]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (64, None, 256)           34304     
                                                                 
 lstm (LSTM)                 (64, None, 1024)          5246976   
                                                                 
 lstm_1 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_2 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_3 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 dense (Dense)               (64, None, 134)           137350    
                                                                 
Total params: 30,596,742
Trainable params: 30,596,742
No

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 134) # (batch_size, sequence_length, vocab_size)


In [21]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 134), dtype=float32, numpy=
array([[ 1.3490765e-05, -2.3079798e-05, -3.7732132e-06, ...,
         5.3216563e-06,  5.1856214e-06, -9.5058385e-06],
       [ 3.2489454e-05, -6.0124396e-05, -8.1154703e-06, ...,
         1.9710275e-05,  2.4158870e-05, -2.5542729e-05],
       [ 3.1502903e-05, -8.2904699e-05, -3.6353260e-05, ...,
         3.9174534e-05,  6.4147134e-05, -3.7721598e-05],
       ...,
       [ 2.7446428e-03,  2.3193609e-03, -1.0971866e-03, ...,
         2.3653975e-03, -5.0335540e-04,  3.6372972e-04],
       [ 2.7111131e-03,  2.2460367e-03, -1.2140614e-03, ...,
         2.2236912e-03, -5.3537416e-04,  4.0792243e-04],
       [ 2.6733731e-03,  2.1493980e-03, -1.2550199e-03, ...,
         2.1046952e-03, -5.1935390e-04,  4.3272472e-04]], dtype=float32)>

In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'дывал на своего подопечного. Деркулов, прихлебывая чай, старательно пялился в окно, за которым с инт'

Next Char Predictions: 
 'eАви4кяпuлхbedъщolОцЖMGNЯйы7r-д…g…lж=уБ5цoISXC 3уXЦhCжлda,»cQQWBиМСтk\tфШЗ\xa0ю;kЛkv(=ч,КС!Рхц Nьl-Пеiв7'


## Train the model

In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 134)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.897507


In [25]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

### Execute the training

In [27]:
EPOCHS = 40

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
50/50 [==============================] - 28s 382ms/step - loss: 3.6154
Epoch 2/40
50/50 [==============================] - 18s 346ms/step - loss: 3.3989
Epoch 3/40
50/50 [==============================] - 19s 368ms/step - loss: 3.3993
Epoch 4/40
50/50 [==============================] - 18s 353ms/step - loss: 3.3995
Epoch 5/40
50/50 [==============================] - 18s 356ms/step - loss: 3.3993
Epoch 6/40
50/50 [==============================] - 19s 364ms/step - loss: 3.3981
Epoch 7/40
50/50 [==============================] - 19s 372ms/step - loss: 3.3962
Epoch 8/40
50/50 [==============================] - 19s 375ms/step - loss: 3.2357
Epoch 9/40
50/50 [==============================] - 21s 408ms/step - loss: 2.8714
Epoch 10/40
50/50 [==============================] - 20s 385ms/step - loss: 2.7239
Epoch 11/40
50/50 [==============================] - 20s 387ms/step - loss: 2.6422
Epoch 12/40
50/50 [==============================] - 20s 388ms/step - loss: 2.5832
Epoch 13/40
5

## Generate text

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_36'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) # перезагрузили модель в батч-сайз 1 (1-01 в лекции)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (1, None, 256)            34304     
                                                                 
 lstm_4 (LSTM)               (1, None, 1024)           5246976   
                                                                 
 lstm_5 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_6 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 lstm_7 (LSTM)               (1, None, 1024)           8392704   
                                                                 
 dense_1 (Dense)             (1, None, 134)            137350    
                                                                 
Total params: 30,596,742
Trainable params: 30,596,742


In [32]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 300

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [33]:
text_ = generate_text(model, start_string=u"Президент заявил об окончании специальной военной операции ")
print(text_)

Президент заявил об окончании специальной военной операции синего на своей подступам к Луганском и станцию и следом с самого подступам голову на свое военной поддержки курса вообще не собирался. Но теперь не было не только в отдел на свое военного представители из самых подствольника и скоторого вместе с остальными середину снайпера в себе со старлей и так 


In [34]:
len(text_)

359

Обучение модели (слои LSTM) на произведении "1984" писателя George Orwell (английский текст):

In [35]:
path_to_file = '/content/1984.txt'

In [36]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 603178 characters


In [37]:
print(text[:500])

Chapter 1
I
t was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his
breast in an effort to escape the vile wind, slipped quickly
through the glass doors of Victory Mansions, though not
quickly enough to prevent a swirl of gritty dust from entering along with him.
The hallway smelt of boiled cabbage and old rag mats. At
one end of it a coloured poster, too large for indoor display,
had been tacked to the wall. It depicted simply an


In [38]:
text = text + text

In [39]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

84 unique characters


In [40]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [41]:
text_as_int, text, len(text_as_int), len(text)

(array([28, 61, 54, ..., 17, 12, 11]),
 'Chapter 1\r\nI\r\nt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith, his chin nuzzled into his\r\nbreast in an effort to escape the vile wind, slipped quickly\r\nthrough the glass doors of Victory Mansions, though not\r\nquickly enough to prevent a swirl of gritty dust from entering along with him.\r\nThe hallway smelt of boiled cabbage and old rag mats. At\r\none end of it a coloured poster, too large for indoor display,\r\nhad been tacked to the wall. It depicted simply an enormous face, more than a metre wide: the face of a man of\r\nabout forty-five, with a heavy black moustache and ruggedly handsome features. Winston made for the stairs. It was\r\nno use trying the lift. Even at the best of times it was seldom working, and at present the electric current was cut\r\noff during daylight hours. It was part of the economy drive\r\nin preparation for Hate Week. The flat was seven flights up,\r\nand Winston, w

### train and target

In [42]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

C
h
a
p
t


In [43]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Chapter 1\r\nI\r\nt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith,'
' his chin nuzzled into his\r\nbreast in an effort to escape the vile wind, slipped quickly\r\nthrough the'
' glass doors of Victory Mansions, though not\r\nquickly enough to prevent a swirl of gritty dust from e'
'ntering along with him.\r\nThe hallway smelt of boiled cabbage and old rag mats. At\r\none end of it a co'
'loured poster, too large for indoor display,\r\nhad been tacked to the wall. It depicted simply an enor'


In [44]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [45]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Chapter 1\r\nI\r\nt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith'
Target data: 'hapter 1\r\nI\r\nt was a bright cold day in April, and the clocks were striking thirteen. Winston Smith,'


In [46]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [47]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [48]:
# def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
#     model = tf.keras.Sequential([
#         tf.keras.layers.Embedding(vocab_size, embedding_dim),

#         tf.keras.layers.GRU(rnn_units,
#                             return_sequences=True,
#                             stateful=False,
#                             recurrent_initializer='glorot_uniform'),

#         tf.keras.layers.GRU(rnn_units,
#                             return_sequences=True,
#                             stateful=False,
#                             recurrent_initializer='glorot_uniform'),

#          tf.keras.layers.GRU(rnn_units,
#                             return_sequences=True,
#                             stateful=False,
#                             recurrent_initializer='glorot_uniform'),

#         tf.keras.layers.Dense(vocab_size)
#     ])
#     return model


def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [49]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [50]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (64, None, 256)           21504     
                                                                 
 lstm_8 (LSTM)               (64, None, 1024)          5246976   
                                                                 
 lstm_9 (LSTM)               (64, None, 1024)          8392704   
                                                                 
 lstm_10 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 lstm_11 (LSTM)              (64, None, 1024)          8392704   
                                                                 
 dense_2 (Dense)             (64, None, 84)            86100     
                                                                 
Total params: 30,532,692
Trainable params: 30,532,692


In [51]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 84) # (batch_size, sequence_length, vocab_size)


In [52]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 84), dtype=float32, numpy=
array([[ 4.14161968e-06, -1.79553062e-05, -1.99345050e-06, ...,
        -1.39920767e-05, -3.94567815e-06,  9.77382842e-06],
       [-2.64481205e-05, -5.77428291e-05, -6.94946493e-06, ...,
        -1.76316116e-05, -3.84255982e-05,  5.87837421e-05],
       [-8.58960630e-05, -1.15724775e-04, -2.05373981e-05, ...,
         2.67248788e-06, -1.35582275e-04,  1.52409950e-04],
       ...,
       [-6.47421344e-04,  2.67686136e-03,  2.88225929e-05, ...,
         1.43137434e-03, -2.24484527e-03,  1.61200063e-03],
       [-6.55299227e-04,  2.64331163e-03, -1.13397575e-04, ...,
         1.47502287e-03, -2.27956520e-03,  1.76390435e-03],
       [-6.02756918e-04,  2.65212916e-03, -2.49956909e-04, ...,
         1.48836989e-03, -2.25117523e-03,  1.87584269e-03]], dtype=float32)>

In [53]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [54]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'at the wall about a metre\r\nabove the level of Winston’s head. He was shoeless; large,\r\ndirty toes we'

Next Char Predictions: 
 'd)BY52J904q6j37IUGF;iXSp\x18mSv.LPD0-\rsG!C‘?Vme\x18OYRFnDGSKQ*08pp7\rZSve!9\r’?JzM]Gl Ezft.Ar8*lz6F-o0]i”IR1'


## Train the model

In [55]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 84)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.430799


In [56]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [57]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*5,
    save_weights_only=True)

### Execute the training

In [58]:
EPOCHS = 40

In [59]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/40
186/186 [==============================] - 81s 390ms/step - loss: 3.1018
Epoch 2/40
186/186 [==============================] - 75s 393ms/step - loss: 2.2028
Epoch 3/40
186/186 [==============================] - 82s 431ms/step - loss: 1.6958
Epoch 4/40
186/186 [==============================] - 77s 404ms/step - loss: 1.4565
Epoch 5/40
186/186 [==============================] - 80s 418ms/step - loss: 1.3225
Epoch 6/40
186/186 [==============================] - 78s 406ms/step - loss: 1.2290
Epoch 7/40
186/186 [==============================] - 77s 404ms/step - loss: 1.1451
Epoch 8/40
186/186 [==============================] - 82s 431ms/step - loss: 1.0631
Epoch 9/40
186/186 [==============================] - 77s 405ms/step - loss: 0.9751
Epoch 10/40
186/186 [==============================] - 83s 434ms/step - loss: 0.8827
Epoch 11/40
186/186 [==============================] - 77s 405ms/step - loss: 0.7813
Epoch 12/40
186/186 [==============================] - 79s 412ms/step - lo

## Generate text

In [60]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_38'

In [61]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1) # перезагрузили модель в батч-сайз 1 (1-01 в лекции)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [62]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (1, None, 256)            21504     
                                                                 
 lstm_12 (LSTM)              (1, None, 1024)           5246976   
                                                                 
 lstm_13 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_14 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 lstm_15 (LSTM)              (1, None, 1024)           8392704   
                                                                 
 dense_3 (Dense)             (1, None, 84)             86100     
                                                                 
Total params: 30,532,692
Trainable params: 30,532,692


In [63]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 300

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.1

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [64]:
text_ = generate_text(model, start_string=u"President announced the end of a special military operation ")
print(text_)

President announced the end of a special military operation at a lower
level for the benefit of the proletariat. There was a whole
chain of separate denounce which existed no longer HAD ONCE existed?
But today, supposing that it could be somehow real power, to turn out more armaments,
to capture more territory, and so on indefinitely. In a little while h


In [65]:
len(text_)

360

Перевод:

Президент объявил об окончании специальной военной операции на более низком уровне в интересах пролетариата. Существовала целая цепочка отдельных доносов, которых больше не существовало, если они когда-то существовали? Но сегодня, предположим, что это могло бы каким-то образом стать реальной силой, создать больше вооружений, захватить больше территории и так далее до бесконечности. Через некоторое время он

Вывод:

Попробовал обучить одну и ту же модель (LSTM) на двух разных текстах на русском и английском языке. По моему мнению во втором случае рекуррентная нейросеть справилась лучше.